### Import the necessary package and functions.
Notice:\
Env: python 3.12.9\
conda create -n AI_property python=3.12.9\
conda init bash\
source ~/.bashrc \
conda activate AI_property

conda install numpy pandas openpyxl openai boto3 -c conda-forge

conda install -c conda-forge fsspec s3fs



In [9]:
# import the necessary packages
import os
import numpy as np
import pandas as pd
import boto3
import time
import sys
from openai import OpenAI
import read_data
import GPT_model

cwd = os.getcwd()
print(cwd)

/home/sagemaker-user/HO_AI_Property_Attribute_2025


### Load the sample data and question file
The sample data is in the directory: 's3://pr-home-datascience/Projects/AdHoc/InternProjects/2025/2025_Summer_AI_Property_Attributes/property_list_NJ_sample.csv'

The question file is in the directory: './question.txt'

In [10]:
# The sample data is the the S3 directory
sample_path = 's3://pr-home-datascience/Projects/AdHoc/InternProjects/2025/2025_Summer_AI_Property_Attributes/property_list_NJ_sample.csv'

# Use the class in read_data.py to load the sample data
df_samples = read_data.DataReader(sample_path)
df_samples.preview()


                address     city state   zip5  zip4 unit_number    county  \
0        340 HOBART AVE  ABSECON    NJ  08201  1127         NaN  ATLANTIC   
1  244 PENNSYLVANIA AVE  ABSECON    NJ  08201  1219         NaN  ATLANTIC   
2     319 E WYOMING AVE  ABSECON    NJ  08201  1711         NaN  ATLANTIC   
3         312 SPRUCE ST  ABSECON    NJ  08201  1726         NaN  ATLANTIC   
4     905 TRAYMORE PKWY  ABSECON    NJ  08201  1513         NaN  ATLANTIC   

  fault_code status_code  delivery_bc      qpid   rundt Number_of_Baths  \
0        NaN      S80000  82011127404  84770770  202410               2   
1        NaN      S80000  82011219448  84770874  202410               2   
2        NaN      S80000  82011711199  84770989  202410               4   
3        NaN      S80000  82011726120  84771105  202410               2   
4        NaN      S80000  82011513054  84771453  202410             1.1   

  Number_of_Bedrooms  
0                  3  
1                  3  
2                

In [11]:
# Load the question file 
# Load the question txt 
print('The promps for the GPT will be as follows:')
with open('questions.txt', 'r', encoding='utf-8') as f:
    questions = np.array(f.read().splitlines(), dtype=str)
for line in questions:
    print(line)


# Load the address txt
# address     city state   zip5  zip4
df_samples.data["full_address"] = df_samples.data["address"] + ", " + df_samples.data["city"] + ", " + df_samples.data["state"] +\
                             ", " + df_samples.data["zip5"] + '-' + df_samples.data["zip4"] 
                            


print('\nThe final questions for the GPT that will be recorded are:')

df_samples.data['questions'] = questions[2] + df_samples.data["full_address"]
print(df_samples.data['questions'][0])
print(df_samples.data['questions'][1])

The promps for the GPT will be as follows:
I will give you the locations, please help me analyze how many bathrooms and bedrooms they have.
Please respond in format: 'Bedrooms: X, Bathrooms: Y.
Can you estimate how many bedrooms and bathrooms are at 

The final questions for the GPT that will be recorded are:
Can you estimate how many bedrooms and bathrooms are at 340 HOBART AVE, ABSECON, NJ, 08201-1127
Can you estimate how many bedrooms and bathrooms are at 244 PENNSYLVANIA AVE, ABSECON, NJ, 08201-1219


### Load the model from GPT_model.py file

1: Initialize OpenAI client with API key\
2: Initialize the bedroom/bathroom estimator with GPT-4.1 model\
3: Fill missing values in sample data with the string 'NaN'\
4: Send general input questions to GPT for the first two samples


In [ ]:

client  = OpenAI(api_key = "")

estimator = GPT_model.BedroomBathroomEstimator(client=client, model="gpt-4.1")
df_samples.data=df_samples.data.fillna('NaN')

# input the general message
for i in range(2):
    estimator.general_input(questions[i])



📝 Added general input: 'I will give you the locations, please help me analyze how many bathrooms and bedrooms they have.'
📝 Added general input: 'Please respond in format: 'Bedrooms: X, Bathrooms: Y.'


### Run the samples: first 1000 samples

1: Define the range of samples to estimate (from n_start to n)\
2: Run GPT estimation on the selected questions and qpids\
3: Print the first few rows of the GPT estimation result


In [13]:
# test sample and input them into model
n = 1000
n_start = 0
df_GPT = estimator.estimate_all(df_samples.data['questions'][n_start: n], df_samples.data['qpid'][n_start: n])

print(df_GPT.head())

Now is the loop 0
Now is the loop 100
Now is the loop 200
Now is the loop 300
Now is the loop 400
Now is the loop 500
Now is the loop 600
Now is the loop 700
Now is the loop 800
Now is the loop 900
                                            question  \
0  Can you estimate how many bedrooms and bathroo...   
1  Can you estimate how many bedrooms and bathroo...   
2  Can you estimate how many bedrooms and bathroo...   
3  Can you estimate how many bedrooms and bathroo...   
4  Can you estimate how many bedrooms and bathroo...   

                                        gpt_response      qpid  bathrooms  \
0                         Bedrooms: 3, Bathrooms: 2.  84770770        2.0   
1                         Bedrooms: 4, Bathrooms: 2.  84770874        2.0   
2                         Bedrooms: 3, Bathrooms: 2.  84770989        2.0   
3  I'm sorry, but I do not have direct access to ...  84771105        2.0   
4                         Bedrooms: 3, Bathrooms: 2.  84771453        2.0   

  

In [14]:
# Show the comsuming results
print(f'The total tokes used are {np.sum(df_GPT['tokens_used'])}.')
print(f'The total time used are {np.sum(df_GPT['time_seconds'])}.')

# Next is to compare the results with the available information, 
# i.e. true bedroom/bathroom number
acc_bed = []
acc_bath = []
acc_all = []
for i in range(n):
    # Match qpid between GPT output and original data
    if df_GPT['qpid'][i] == df_samples.data['qpid'][i]:
        pred_bed = df_GPT['bedrooms'][i]
        pred_bath = df_GPT['bathrooms'][i]
        true_bed = float(df_samples.data['Number_of_Bedrooms'][i] )
        true_bath = float(df_samples.data['Number_of_Baths'][i])

        # Check if predicted bedroom count is correc
        if pred_bed == true_bed:
            acc_bed.append({
                'i': i,
                'qpid': df_GPT['qpid'][i],
                'Number_of_Bedrooms': true_bed
            })

        # Check if predicted bathroom count is correct
        if pred_bath == true_bath:
            acc_bath.append({
                'i': i,
                'qpid': df_GPT['qpid'][i],
                'Number_of_Baths': true_bath
            })

        # Check if both are correct
        if pred_bed == true_bed and pred_bath == true_bath:
            acc_all.append({
                'i': i,
                'qpid': df_GPT['qpid'][i],
                'Number_of_Bedrooms': true_bed,
                'Number_of_Baths': true_bath
            })

names = ['bathroom', 'bedroom', 'both bathroom and bedroom']
for acc, name in zip([acc_bath, acc_bed, acc_all], names):
    print(f'There are {len(acc)} samples have the same {name} number.')
    print(f'The accuracy is {len(acc)/n:.2%}.')

The total tokes used are 114393.
The total time used are 1322.08.
There are 334 samples have the same bathroom number.
The accuracy is 33.40%.
There are 440 samples have the same bedroom number.
The accuracy is 44.00%.
There are 184 samples have the same both bathroom and bedroom number.
The accuracy is 18.40%.


### Save data to csv file, the whole data and correct predictions

In [15]:
# Next is to output the datafram into the csv file
def merge_predictions_with_samples(df_samples, df_GPT, same_results=None):
    """
    Merge df_samples.data with GPT predictions from df_GPT
    based on row index (i) of same_results. 
    If `same_results` is provided, only merge rows whose indices are listed in same_results['i'].
    If `same_results` is None, merge all rows based on row index.

    Args:
        df_samples: Data container with `.data` attribute as a DataFrame (original sample data).
        df_GPT: DataFrame with GPT outputs (must include 'qpid', 'bedrooms', 'bathrooms', etc.).
        same_results: Optional list of dicts containing 'i' indices to subset matching rows.

    Output: includes all columns from df_samples, and adds four prediction columns:
    - predicted_bedroom
    - predicted_bathroom
    - tokens_used
    - time_seconds
    """

    merged_records = []
    # Determine how many rows to iterate over
    length = len(df_GPT)
    if same_results:
        length = len(same_results)
    for j in range(length):
        i = j
        # Determine actual index from same_results or fallback to j
        if same_results:
            i = same_results['i'][j]
        
        # Skip if qpid mismatch (sanity check)
        if df_samples.data['qpid'][i] != df_GPT['qpid'][i]:
            continue  # skip mismatches

        # Convert original row to dict (drop last two columns)
        row = df_samples.data.iloc[i,:-2].to_dict()

        row['predicted_bathroom'] = df_GPT['bathrooms'][i]
        row['predicted_bedroom'] = df_GPT['bedrooms'][i]
        row['tokens_used'] = df_GPT['tokens_used'][i]
        row['time_seconds'] = df_GPT['time_seconds'][i]

        merged_records.append(row)

    return pd.DataFrame(merged_records)

'''
## You can save the data with correct bathroom/bedroom data using the following codes
same_all = pd.DataFrame(acc_all)
same_bath = pd.DataFrame(acc_bath)
same_bed = pd.DataFrame(acc_bed)

results_all = merge_predictions_with_samples(df_samples, df_GPT, same_all)
results_bath =  merge_predictions_with_samples(df_samples, df_GPT, same_bath)
results_bed =  merge_predictions_with_samples(df_samples, df_GPT, same_bed)

results_all.to_excel(f'./results/results_all_n{n}.xlsx', index=False)
results_bath.to_excel(f'./results/results_bath_n{n}.xlsx', index=False)
results_bed.to_excel(f'./results/results_bed_n{n}.xlsx', index=False)
print(f'Save files successfully')
'''

# The following is to save all the data for the further comparison
data_all = merge_predictions_with_samples(df_samples, df_GPT, same_results=None)
data_all.to_excel(f'./results/data_all_n{n}.xlsx', index=False)




In [16]:
# This part is to modify the width of the Excel cell to make it more clear
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
'''
excel_paths = [
    f'./results/results_all_n{n}.xlsx',
    f'./results/results_bath_n{n}.xlsx',
    f'./results/results_bed_n{n}.xlsx',
]
'''
excel_paths = [
    f'./results/data_all_n{n}.xlsx'
]

for excel_path in excel_paths:
    wb = load_workbook(excel_path)
    sheet_name = wb.sheetnames[0]  # ✅ use the first sheet name (or change as needed)
    ws = wb[sheet_name] if sheet_name else wb.active

    for col in ws.columns:
        max_len = 0
        col_letter = get_column_letter(col[0].column)
        for cell in col:
            try:
                val_len = len(str(cell.value))
                max_len = max(max_len, val_len)
            except:
                pass
        ws.column_dimensions[col_letter].width = max_len * 1.2

    wb.save(excel_path)
    print('The Excel is successfully modified')

The Excel is successfully modified
